In [9]:
import argparse
import sys
import tensorflow as tf
sys.path.append('./code')
sys.path.append('./code/optimization')

from optimization.optimize import build_tensorflow
from common import settings_reader, io, model_builder, optimizer_parameter_parser, evaluation, auxilliaries
import numpy as np

setting_file = './settings/gcn_basis.exp'
dataset = './data/Toy/'
settings = settings_reader.read(setting_file)
print(settings)

{'Shared': {'CodeDimension': '500'}, 'Optimizer': {'EarlyStopping': {'BurninPhaseDuration': '6000', 'CheckEvery': '100'}, 'ReportTrainLossEvery': '100', 'Algorithm': {'learning_rate': '0.01', 'Name': 'Adam'}, 'MaxGradientNorm': '1'}, 'General': {'ExperimentName': 'models/GcnBlock', 'GraphSplitSize': '0.5', 'NegativeSampleRate': '10'}, 'Evaluation': {'Metric': 'MRR'}, 'Encoder': {'NumberOfLayers': '2', 'StoreEdgeData': 'No', 'UseInputTransform': 'Yes', 'SkipConnections': 'None', 'Name': 'gcn_basis', 'UseOutputTransform': 'No', 'RandomInput': 'No', 'NumberOfBasisFunctions': '5', 'AddDiagonal': 'No', 'Concatenation': 'No', 'DropoutKeepProbability': '0.8', 'PartiallyRandomInput': 'No', 'InternalEncoderDimension': '500', 'DiagonalCoefficients': 'No'}, 'Decoder': {'Name': 'bilinear-diag', 'RegularizationParameter': '0.01'}}


In [10]:
'''
1. Load datasets
'''
relations_path = dataset + '/relations.dict'
entities_path = dataset + '/entities.dict'
train_path = dataset + '/train.txt'
valid_path = dataset + '/valid.txt'
test_path = dataset + '/test.txt'

#Extend paths for accuracy evaluation:
if settings['Evaluation']['Metric'] == 'Accuracy':
    valid_path = dataset + '/valid_accuracy.txt'
    test_path = dataset + '/test_accuracy.txt'

train_triplets = io.read_triplets_as_list(train_path, entities_path, relations_path)
valid_triplets = io.read_triplets_as_list(valid_path, entities_path, relations_path)
test_triplets = io.read_triplets_as_list(test_path, entities_path, relations_path)

train_triplets = np.array(train_triplets)
valid_triplets = np.array(valid_triplets)
test_triplets = np.array(test_triplets)

entities = io.read_dictionary(entities_path)
relations = io.read_dictionary(relations_path)

#print(train_triplets)

In [3]:
'''
2. Load general settings
'''
encoder_settings = settings['Encoder']
decoder_settings = settings['Decoder']
shared_settings = settings['Shared']
general_settings = settings['General']
optimizer_settings = settings['Optimizer']
evaluation_settings = settings['Evaluation']

general_settings.put('EntityCount', len(entities))
general_settings.put('RelationCount', len(relations))
general_settings.put('EdgeCount', len(train_triplets))

encoder_settings.merge(shared_settings)
encoder_settings.merge(general_settings)
decoder_settings.merge(shared_settings)
decoder_settings.merge(general_settings)

optimizer_settings.merge(general_settings)
evaluation_settings.merge(general_settings)

In [8]:
'''
3. Construct the encoder-decoder pair:
'''
# common.model_builder.build_encoder() -> extras.graph_representations.Representation -> encoders.affine_transform.AffineTransform ->
#      common.model_builder.apply_basis_gcn() -> encoders.relation_embedding.RelationEmbedding
encoder = model_builder.build_encoder(encoder_settings, train_triplets) # RelationEmbedding object inheriting from Model

# decoders.bilinear_diag.BilinearDiag object inheriting from Model
model = model_builder.build_decoder(encoder, decoder_settings) # next_component = encoder

print(encoder.needs_graph()) 
print(type(encoder), type(model))

print('--------')
print(model.get_graph().get_sender_indices())
print(model.get_graph().get_receiver_indices())

---------------------------------------------------------------
func: extras/graph_representations.Representation.__init__
  entity_count: 16
  relation_count: 9
  edge_count: 86
---------------------------------------------------------------
---------------------------------------------------------------
common/model_builder.build_encoder: gcn_basis
  input shape: [16, 500]
  internal shape: [500, 500]
  projection shape: [500, 500]
  relation shape: [16, 500]
  layers: 2
---------------------------------------------------------------
---------------------------------------------------------------
common/model_builder.build_encoder: gcn_basis
layer type: <class 'encoders.message_gcns.gcn_basis.BasisGcn'> gcn_id: 1
layer type: <class 'encoders.message_gcns.gcn_basis.BasisGcn'> gcn_id: 0
layer type: <class 'encoders.affine_transform.AffineTransform'>
---------------------------------------------------------------
True
<class 'encoders.relation_embedding.RelationEmbedding'> <class 'decod

ValueError: Tried to convert 'input' to a tensor and failed. Error: None values not supported.

In [6]:
'''
4. Construct the optimizer with validation MRR as early stopping metric:
'''
opp = optimizer_parameter_parser.Parser(optimizer_settings)
opp.set_save_function(model.save) # BilinearDiag -> Model.save

scorer = evaluation.Scorer(evaluation_settings)
scorer.register_data(train_triplets)
scorer.register_data(valid_triplets)
scorer.register_data(test_triplets)
scorer.register_degrees(train_triplets)
scorer.register_model(model)
scorer.finalize_frequency_computation(np.concatenate((train_triplets, valid_triplets, test_triplets), axis=0))
print('---------------------------------------------------------------')
print('  train -> step 4 -> scorer(evaluation.Scorer)')
#print('  known_subject_triples', scorer.known_subject_triples)
#print('  known_object_triples', scorer.known_object_triples)
#print('  relation_freqs', scorer.relation_freqs)
#print('  avg_freq', scorer.avg_freq)
print('---------------------------------------------------------------')

def score_validation_data(validation_data):
    score_summary = scorer.compute_scores(validation_data, verbose=False).get_summary()
    #score_summary.dump_degrees('dumps/degrees.in', 'dumps/degrees.out')
    #score_summary.dump_frequencies('dumps/near.freq', 'dumps/target.freq')
    #score_summary.pretty_print()

    if evaluation_settings['Metric'] == 'MRR':
        lookup_string = score_summary.mrr_string()
    elif evaluation_settings['Metric'] == 'Accuracy':
        lookup_string = score_summary.accuracy_string()

    early_stopping = score_summary.results['Filtered'][lookup_string]

    score_summary = scorer.compute_scores(test_triplets, verbose=False).get_summary()
    score_summary.pretty_print()

    return early_stopping

opp.set_early_stopping_score_function(score_validation_data)

adj_list = [[] for _ in entities]
for i,triplet in enumerate(train_triplets):   # undirected graph ?
    adj_list[triplet[0]].append([i, triplet[2]])
    adj_list[triplet[2]].append([i, triplet[0]])
degrees = np.array([len(a) for a in adj_list])
adj_list = [np.array(a) for a in adj_list]
print(adj_list)
print(degrees)

# not in use
def sample_TIES(triplets, n_target_vertices):
    vertex_set = set([])

    edge_indices = np.arange(triplets.shape[0])
    while len(vertex_set) < n_target_vertices:
        edge = triplets[np.random.choice(edge_indices)]
        new_vertices = [edge[0], edge[1]]
        vertex_set = vertex_set.union(new_vertices)

    sampled = [False]*triplets.shape[0]

    for i in edge_indices:
        edge = triplets[i]
        if edge[0] in vertex_set and edge[2] in vertex_set:
            sampled[i] = True

    return edge_indices[sampled]


def sample_edge_neighborhood(triplets, sample_size):

    edges = np.zeros((sample_size), dtype=np.int32)

    #initialize
    sample_counts = np.array([d for d in degrees])
    picked = np.array([False for _ in triplets])
    seen = np.array([False for _ in degrees])

    for i in range(0, sample_size):
        weights = sample_counts * seen

        if np.sum(weights) == 0:
            weights = np.ones_like(weights)
            weights[np.where(sample_counts == 0)] = 0

        probabilities = (weights) / np.sum(weights)
        chosen_vertex = np.random.choice(np.arange(degrees.shape[0]), p=probabilities)
        chosen_adj_list = adj_list[chosen_vertex]
        seen[chosen_vertex] = True

        chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
        chosen_edge = chosen_adj_list[chosen_edge]
        edge_number = chosen_edge[0]

        while picked[edge_number]:
            chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
            chosen_edge = chosen_adj_list[chosen_edge]
            edge_number = chosen_edge[0]

        edges[i] = edge_number
        other_vertex = chosen_edge[1]
        picked[edge_number] = True
        sample_counts[chosen_vertex] -= 1
        sample_counts[other_vertex] -= 1
        seen[other_vertex] = True

    return edges


if 'NegativeSampleRate' in general_settings:
    ns = auxilliaries.NegativeSampler(int(general_settings['NegativeSampleRate']), general_settings['EntityCount'])
    ns.set_known_positives(train_triplets)

    def t_func(x): #horrible hack!!!
        arr = np.array(x)
        #RelationEmbedding <- model.needs_graph() <- RelationEmbedding.next_component.needs_graph() <- 
        #    BasisGcn <- MessageGcn.needs_graph: True
        if not encoder.needs_graph():
            return ns.transform(arr)
        else:
            if 'GraphBatchSize' in general_settings:
                graph_batch_size = int(general_settings['GraphBatchSize'])

                '''
                n = np.zeros(100)
                for i in range(100):
                    if i % 20 == 0:
                        print(i)
                    n[i] = sample_TIES(arr, 1000).shape[0]

                print(n.mean())
                print(n.std())
                exit()
                '''
                
                #graph_batch_ids = sample_TIES(arr, 1000) #sample_edge_neighborhood(arr, graph_batch_size)
                graph_batch_ids = sample_edge_neighborhood(arr, graph_batch_size)
            else:
                graph_batch_size = arr.shape[0]
                graph_batch_ids = np.arange(graph_batch_size)

            graph_batch = np.array(train_triplets)[graph_batch_ids]

            # Apply dropouts:
            graph_percentage = float(general_settings['GraphSplitSize'])
            split_size = int(graph_percentage * graph_batch.shape[0])
            graph_split_ids = np.random.choice(graph_batch_ids, size=split_size, replace=False)
            graph_split = np.array(train_triplets)[graph_split_ids]

            t = ns.transform(graph_batch)

            if 'StoreEdgeData' in encoder_settings and encoder_settings['StoreEdgeData'] == "Yes":
                return (graph_split, graph_split_ids, t[0], t[1])
            else:
                return (graph_split, t[0], t[1])

    opp.set_sample_transform_function(t_func)

---------------------------------------------------------------
  train -> step 4 -> scorer(evaluation.Scorer)
---------------------------------------------------------------
[array([[18,  9],
       [22,  6],
       [23, 15],
       [24,  8],
       [25,  2],
       [26,  3],
       [27,  9]]), array([[17,  9],
       [28,  3],
       [29,  9]]), array([[21,  9],
       [25,  0],
       [35,  6],
       [40,  5]]), array([[ 0, 10],
       [ 3, 12],
       [14,  9],
       [26,  0],
       [28,  1],
       [30,  6],
       [39,  5]]), array([[ 4, 12],
       [ 9,  9],
       [10, 13],
       [11, 12],
       [12,  6],
       [13, 15]]), array([[33,  6],
       [37,  8],
       [38,  6],
       [39,  3],
       [40,  2]]), array([[ 7, 12],
       [12,  4],
       [16,  9],
       [22,  0],
       [30,  3],
       [31,  7],
       [32, 15],
       [33,  5],
       [34,  8],
       [35,  2],
       [36, 15],
       [38,  5],
       [42, 11]]), array([[15,  9],
       [31,  6]]), array([[2

In [23]:
'''
5. Initialize for training:
'''

# Hack for validation evaluation:
model.preprocess(train_triplets)  # decoders.bilinear_diag.BilinearDiag <- Model.preprocess
model.register_for_test(train_triplets) # call: Model.register_for_test

model.initialize_train()  # decoders/bilinear_diag.initialize_train

optimizer_weights = model.get_weights()
for weight in optimizer_weights:
    print(weight.get_shape)
'''
weight order
  AffineTransform.local_get_weights: [self.W, self.b]
  BasisGcn.local_get_weights: [self.W_forward, self.W_backward, self.C_forward, self.C_backward, self.W_self, self.b]
  BasisGcn.local_get_weights: [self.W_forward, self.W_backward, self.C_forward, self.C_backward, self.W_self, self.b]
  RelationEmbedding.local_get_weights: [self.W_relation]
'''    
    
optimizer_input = model.get_train_input_variables() #After model.initialize_train() 
for weight in optimizer_input:
    print(weight.get_shape)
'''
optimizer_input order
  extras/graph_representations.Representation.local_get_train_input_variables: [self.X]
  decoders/bilinear_diag.local_get_train_input_variables.local_get_train_input_variables: [self.X, self.Y]
'''

# model.get_loss()
# decoders/bilinear_diag.BilinearDiag.get_loss() -> BilinearDiag.compute_codes() -> RelationEmbedding.get_all_codes() ->
#      BasisGcn (MessageGcn.get_all_codes()) (两层) -> AffineTransform.get_all_codes()
loss = model.get_loss(mode='train') + model.get_regularization()

15
(16, 500)
(500,)
(500, 5, 500)
(500, 5, 500)
(9, 5)
(9, 5)
(500, 500)
(500,)
(500, 5, 500)
(500, 5, 500)
(9, 5)
(9, 5)
(500, 500)
(500,)
(16, 500)
[<tf.Tensor 'graph_edges_1:0' shape=(?, 3) dtype=int32>, <tf.Tensor 'Placeholder_3:0' shape=(?, 3) dtype=int32>, <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=float32>]


In [11]:
'''
6. Clean this shit up:
'''

for add_op in model.get_additional_ops():
    opp.additional_ops.append(add_op)

optimizer_parameters = opp.get_parametrization()

In [12]:
'''
7. Train with Converge:
'''

model.session = tf.Session()
print('build tensorflow...')
optimizer = build_tensorflow(loss, optimizer_weights, optimizer_parameters, optimizer_input)
optimizer.set_session(model.session)
print('fit...')
optimizer.fit(train_triplets, validation_data=valid_triplets)
#scorer.dump_all_scores(valid_triplets, 'dumps/subjects.valid', 'dumps/objects.valid')
#scorer.dump_all_scores(test_triplets, 'dumps/subjects.test', 'dumps/objects.test')

build tensorflow...
SampleTransformer
GradientClipping
Adam
TrainLossReporter
EarlyStopper
ModelSaver
fit...
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initial loss: 24.8577
Average train loss for iteration 1-100: 34.178371408
Average train loss for iteration 101-200: 0.27878484413
Average train loss for iteration 201-300: 0.228052216172
Average train loss for iteration 301-400: 0.197706042677
Average train loss for iteration 401-500: 0.188237770945
Average train loss for iteration 501-600: 0.185940514207
Average train loss for iteration 601-700: 0.175802872181
Average train loss for iteration 701-800: 0.17519323796
Average train loss for iteration 801-900: 0.175423926711
Average train loss for iteration 901-1000: 0.16630381912
Average train loss for iteration 1001-1100: 0.163117232621
Average train loss for iteration 1101-1200: 0.160824085176
Average train loss for iteration 1201-1300: 0.162100131214
Average train loss for iteration 1301-1400: 0.16184578